In [93]:
import librosa 
from scipy.io import wavfile as wav
import numpy as np
import keras

In [94]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
model = keras.models.load_model('saved_models/weights.best.basic_mlp.hdf5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [95]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import numpy as np

# Convert features and corresponding classification labels into numpy arrays
X = np.load("/home/ubuntu/npy_files/X.npy")
y = np.load("/home/ubuntu/npy_files/y.npy")

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

In [96]:
def extract_feature(file_name):
   
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    return np.array([mfccsscaled])

In [97]:
def print_class(file_name):
    prediction_feature = extract_feature(file_name) 

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )
    return predicted_class[0]

In [98]:
def print_noise_result(file_name):
    prediction_feature = extract_feature(file_name) 

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    if predicted_class[0] == 'children_playing':
        noise_result = 'noise'
    elif predicted_class[0] == 'siren':
        noise_result = 'siren'
    else:
        noise_result = 'etc'
    
    print("The predicted class is:", predicted_class[0], '\n') 
    print("The noise result is:", noise_result, '\n') 

#    predicted_proba_vector = model.predict_proba(prediction_feature) 
 #   predicted_proba = predicted_proba_vector[0]
  #  for i in range(len(predicted_proba)): 
   #     category = le.inverse_transform(np.array([i]))
    #    print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )
        
    return noise_result

In [99]:
# Class: test dog

filename = '/home/ubuntu/wav/sound.wav' 
print_class(filename)

The predicted class is: children_playing 

air_conditioner 		 :  0.00155912176705896854400634765625
car_horn 		 :  0.00000041178518017659371253103018
children_playing 		 :  0.88746505975723266601562500000000
dog_bark 		 :  0.01371119543910026550292968750000
drilling 		 :  0.00013054620649199932813644409180
engine_idling 		 :  0.00000175551883785374229773879051
gun_shot 		 :  0.09615865349769592285156250000000
jackhammer 		 :  0.00000000108301601020599491675966
siren 		 :  0.00028599920915439724922180175781
street_music 		 :  0.00068734365049749612808227539062


'children_playing'

In [100]:
import os.path, time
class_name = print_class(filename)
noise_result = print_noise_result(filename)
time = time.ctime(os.path.getmtime(filename))

print(class_name)
print(noise_result)
print(time)

The predicted class is: children_playing 

air_conditioner 		 :  0.00155912176705896854400634765625
car_horn 		 :  0.00000041178518017659371253103018
children_playing 		 :  0.88746505975723266601562500000000
dog_bark 		 :  0.01371119543910026550292968750000
drilling 		 :  0.00013054620649199932813644409180
engine_idling 		 :  0.00000175551883785374229773879051
gun_shot 		 :  0.09615865349769592285156250000000
jackhammer 		 :  0.00000000108301601020599491675966
siren 		 :  0.00028599920915439724922180175781
street_music 		 :  0.00068734365049749612808227539062
The predicted class is: children_playing 

The noise result is: noise 

children_playing
noise
Tue Jan 21 11:37:39 2020


In [101]:
import csv
with open('/home/ubuntu/noisedata.csv', 'a') as newFile:
    newFileWriter = csv.writer(newFile)
    newFileWriter.writerow(['용산구12-1891', noise_result, class_name, time])

In [102]:
!cat /home/ubuntu/noisedata.csv